In [1]:
# -*- coding: utf-8 -*-
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import unittest, time, re
import csv
import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

In [12]:
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "localhost:9222")
chrome_driver = "/home/kimjadong2020/Documents/chromedriver"
driver = webdriver.Chrome(chrome_driver, options=chrome_options)

In [3]:
def refillGold():
    url = "https://rivalregions.com/#parliament/offer"
    driver.get(url)
    driver.refresh()
    time.sleep(5)
    driver.find_element_by_xpath("//div[@id='offer_dd']/div/div/div").click()
    time.sleep(4)
    driver.find_element_by_link_text("Resources exploration: state").click()
    driver.find_element_by_id("offer_do").click()
    
def goldRefiiler(): # 금 채우기 검사 및 실행
    url = "https://rivalregions.com/#listed/stateresources/3330"
    driver.get(url)
    driver.refresh()
    time.sleep(3)

    gold_reserve = driver.find_elements_by_class_name("list_level")

    tempList = []
    for element in gold_reserve:
        tempList.append(float(element.text))

    array_gold = np.array(tempList).reshape(-1, 4)

    df_gold = pd.DataFrame(
        array_gold, 
       columns=["Explored", "Maximum", "Deep_exploration", "Limit_left"]
    )

    str_expr = "Limit_left > 0 and Explored < Maximum - 50"
    df_q = df_gold.query(str_expr) 

    if df_q.empty:
        print("채우지 않음")
    else:
        print("do refill")
        refillGold()

def budgetCheck():
    url = "https://rivalregions.com/#state/details/3330"
    driver.get(url)
    driver.refresh()
    time.sleep(4)

    itemList = []

    budgetList = driver.find_element_by_css_selector(
        "#header_slide_inner > div.minwidth > div.slide_profile_photo > div.imp").find_elements_by_class_name("tip")

    for i in budgetList:
        item = int(i.text.split(" ")[0].replace(".",""))
        itemList.append(item)

    df = pd.DataFrame(
        np.array(itemList).reshape(1, 6), 
        columns=["cash", "gold", "oil", "ore", "uranium", "diamond"]
    ).T
    return df

In [4]:
def csvAppend (priceList):
    with open("price_list.csv", "a", newline='') as f:
        writer = csv.writer(f)
        writer.writerow( priceList )

def priceLoging (numList):
    priceList = []
    now = str( datetime.datetime.now() )
    priceList.append(now)
    
    for num in numList:
        s_xpath = "//div[@url='"+ str(num) +"']"
        driver.find_element_by_xpath( s_xpath ).click() # 자원 클릭
        time.sleep(3.5)
        element_price = driver.find_element_by_xpath("/html/body/div[6]/div[1]/div[1]/div[2]/div[1]/div[3]/span/span")
        time.sleep(1)
        price = int(element_price.text.split(" ")[0].replace(".",""))
        priceList.append(price)
        
    csvAppend( priceList )

In [5]:
def getURL(url):
    driver.get(url)
    driver.implicitly_wait(50)
    driver.refresh()

In [6]:
getURL("https://rivalregions.com/#overview")

In [11]:
itemList = [3, 4, 11, 15, 26] # 석유 광물 우라늄 다이아 라이벌륨

In [ ]:
def work():    
    getURL("https://rivalregions.com/#work")
    xpath = "//*[@id='content']/div[6]/div[2]/div[2]/div[3]/div[1]/span/span[1]"
    driver.find_element_by_xpath(xpath).click()
    driver.find_element_by_id("header_my_fill_bar").click()
    driver.refresh()
    driver.find_element_by_xpath(xpath).click()

In [9]:
def controller():   
    # price logging
    getURL("https://rivalregions.com/#storage")
    priceLoging(itemList)
    
    work()    

In [19]:
# war train
getURL("https://rivalregions.com/#war")
driver.find_element_by_xpath("//div[@id='content']/div[4]/div[2]/div").click()
driver.find_element_by_id("war_my_alpha").click()
driver.refresh()
driver.find_element_by_id("header_my_fill_bar").click()    
driver.find_element_by_id("war_my_alpha").click()

In [20]:
controller()

AttributeError: 'NoneType' object has no attribute 'click'

In [14]:
# control plane
for i in range(24):
    for j in range(5):
        controller()
        time.sleep(590)

ElementClickInterceptedException: Message: element click intercepted: Element <span class="work_how_many_f">...</span> is not clickable at point (731, 365). Other element would receive the click: <div class="work_results2" style="font-size: 15px !important;">...</div>
  (Session info: chrome=86.0.4240.75)


In [25]:
# check budget and refill gold
goldRefiiler()

do refill


In [11]:
# csv HEADER
with open("price_list.csv", "w", newline='') as f:
    writer = csv.writer(f)
    column = ["time", "oil", "ore", "uranium", "diamonds", "rivalium"]
    writer.writerow( column )

In [125]:
s_xpath = "//div[@url='26']"
driver.find_element_by_xpath( s_xpath ).click() # 자원 클릭
element_price = driver.find_element_by_xpath("/html/body/div[6]/div[1]/div[1]/div[2]/div[1]/div[3]/span/span")
price = int(element_price.text.split(' ')[0].replace(".",""))        
print(price)

169990


In [18]:
#parliament_active_laws > div > div > div:nth-child(3) > div.float_left.parliament_sh1.small

In [ ]:
# 군사학교
selector = "#quests_wrap > div.tip.hov.pointer.quest_have_perk_ma"

driver.find_element_by_css_selector(selector).click()

driver.find_element_by_xpath("//div[@id='quests_wrap']/div").click()

In [ ]:
# check budget and refill gold
df_previous_budget = budgetCheck()
goldRefiiler()
df_budget = budgetCheck()
df_diff = df_budget - df_previous_budget

In [116]:
print(df_diff)

In [16]:
df_diff

,0
cash,0
gold,0
oil,0
ore,0
uranium,0
diamond,0


In [121]:
df2_diff

,0
cash,138386499
gold,213539386
oil,0
ore,0
uranium,0
diamond,0


In [34]:
# 홈 버튼
driver.find_element_by_class_name("header_logo_inner").click()

driver = webdriver.Chrome()

In [8]:
# 총 재산 체크
money = driver.find_element_by_xpath("/html/body/div[5]/div[1]/div[1]/div[4]/span[1]/span").text
print(f"총 재산 : {money}")
print(money.replace(".","",4))

총 재산 : 658.487.849
658487849


In [22]:
req = requests.get(url)

In [23]:
html = req.text

In [24]:
with open("test.html", "w", newline='', encoding="utf-8") as f:
    f.write( html )